In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torchvision
import torchvision.transforms as transforms

from PIL import Image

img_dir = "images/train/"

csv_file = "products.csv"

products = pd.read_csv("products.csv")

# label list
labels_all = list(np.unique(products['GS1 Form'])) + list(np.unique(products['Material'])) + list(np.unique(products['Colour']))
label_dict = { i:val for i, val in enumerate(labels_all)}

# Define relevant variables for the ML task
batch_size = 32
num_classes = len(labels_all)
learning_rate = 0.01
num_epochs = 10

# normalizing data ...
# transform = transforms.Compose([transforms.Resize((32,32)),
#                                      transforms.ToTensor(),
#                                      transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
#                                                           std=[0.2023, 0.1994, 0.2010])
#                                      ])

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Device will determine whether to run the training on GPU or CPU.
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
"""Loading data with PyTorch"""

from torch.utils.data import DataLoader, Dataset
import random
import os

class MultiLabelDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.img_paths = [str(barcode) + '.jpg' for barcode in self.dataframe['Barcode'].values]
        self.labels = {'GS1 Form':self.dataframe['GS1 Form'].values, 'Material':self.dataframe['Material'].values, 'Colour':self.dataframe['Colour'].values}
        
        # dictionary for each class
        self.gs1_form = { val:i for i, val in enumerate(list(set(self.labels['GS1 Form'])))}
        self.material = { val:i for i, val in enumerate(list(set(self.labels['Material'])))}
        self.colour = { val:i for i, val in enumerate(list(set(self.labels['Colour'])))}
        
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_paths[idx])
        img = Image.open(img_path).convert('RGB')
        
        # num_classes = len(self.labels_all)
        
        label1 = torch.nn.functional.one_hot(torch.tensor(self.gs1_form[self.labels['GS1 Form'][idx]]), num_classes = len(self.gs1_form))
        label2 = torch.nn.functional.one_hot(torch.tensor(self.material[self.labels['Material'][idx]]), num_classes = len(self.material))
        label3 = torch.nn.functional.one_hot(torch.tensor(self.colour[self.labels['Colour'][idx]]), num_classes = len(self.colour))
        
        label = torch.cat((label1, label2, label3))
        
        # label = torch.nn.functional.one_hot(label, num_classes = num_classes)
        
        if self.transform:
            img = self.transform(img)
        return img, label

class MultiLabelDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=batch_size, shuffle=False, num_workers=0):
        super(MultiLabelDataLoader, self).__init__(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    def __iter__(self):
        for i, (data, label) in enumerate(super().__iter__()):
            yield (data, label)

    def __len__(self):
        return len(self.dataset)  

# Create an instance of the custom dataset class
train_dataset = MultiLabelDataset(img_dir, csv_file, transform=transform)

# Create an instance of the custom data loader class
# train_loader = MultiLabelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# data_iter = iter(train_loader)
# images, labels = next(data_iter)

train_dataset[0]

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]))

In [33]:
# plt.imshow(images[0].permute(1, 2, 0))

# val, indexes = torch.max(labels[0], dim=1)

# print(train_dataset.R_gs1_form[int(indexes[0])])
# print(train_dataset.R_material[int(indexes[1])])
# print(train_dataset.R_colour[int(indexes[2])])

In [ ]:
# Define a CNN model
class MultiLabelCNN(torch.nn.Module):
    def __init__(self):
        super(MultiLabelCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = torch.nn.Linear(4096, 128)
        self.fc2 = torch.nn.Linear(128, 21)
        
    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.sigmoid(self.fc2(x))
        return x

# Initialize model, loss function and optimizer
model = MultiLabelCNN()
criterion = torch.nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters())

from tqdm.notebook import tqdm

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(tqdm(train_loader, desc=(f'Epoch {epoch + 1}'))):
        # Forward pass
        outputs = model(images)
        # labels = labels.view(-1, 21)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Test the model
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Epoch 1:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch [1/10], Loss: 0.6989


Epoch 2:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch [2/10], Loss: 0.6989


Epoch 3:   0%|          | 0/313 [00:00<?, ?it/s]

...

Epoch 6:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch [6/10], Loss: 0.6989


Epoch 7:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch [7/10], Loss: 0.6989


Epoch 8:   0%|          | 0/313 [00:00<?, ?it/s]